### This notebook will take the cnn_wide.csv dataset and create a similar one that makes it take into account the previous second.  240xN --> 480xN

In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf

import sys
sys.path.insert(0, '../')

import script as sc

In [7]:
tf.logging.set_verbosity(tf.logging.INFO)

### Data Import

In [32]:
# data = pd.read_csv(r"C:\Users\yashm\Google Drive\Data Capstone_\Project Folder\PreWideData\cnn_wide.csv")
data = pd.read_csv(r'C:/Users/Hans/Documents/CalPoly4thYear/Data451/cnn_wide.csv')
data.shape

(116090, 243)

In [33]:
data.head()

,time,0,1,2,3,4,5,6,7,8,...,232,233,234,235,236,237,238,239,labels,activity
0,0.0,-0.605,0.617,0.070,-0.621,0.602,0.023,-0.605,0.590,-0.016,...,0.934,-0.133,-0.609,0.918,-0.191,-0.602,0.891,-0.195,0,0
1,1.0,-0.512,0.914,-0.191,-0.441,0.918,-0.152,-0.391,0.914,-0.102,...,0.723,0.613,-0.680,0.727,0.598,-0.680,0.719,0.586,0,0
2,2.0,-0.672,0.711,0.586,-0.648,0.684,0.582,-0.621,0.668,0.574,...,0.359,0.473,-0.867,0.359,0.434,-0.895,0.363,0.422,0,0
3,3.0,-0.898,0.379,0.426,-0.879,0.387,0.441,-0.859,0.402,0.445,...,0.402,0.465,-0.875,0.398,0.445,-0.859,0.391,0.441,0,0
4,4.0,-0.848,0.395,0.441,-0.848,0.379,0.438,-0.848,0.367,0.426,...,0.496,0.418,-0.801,0.496,0.422,-0.801,0.492,0.422,0,0


In [34]:
## transform into 160x3 columns (create lag)
data[np.arange(240,480).astype(str)] = data[np.arange(240).astype(str)].shift(-1)
data[['time','activity','labels']] = data[['time','activity','labels']].shift(-1)

In [48]:
# for c in data.columns:
#     print (c)
data['labels'].value_counts()

0.0    64311
3.0    22966
2.0    13337
1.0     8557
6.0     5651
4.0      840
5.0      427
Name: labels, dtype: int64

In [49]:
data = data.drop(data.index[len(data) - 1])

In [ ]:
train = data.sample(int(len(data)*.7))
test = data.drop(train.index)

In [66]:
np.arange(480)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [68]:
labels = train['activity']
K = len(train['activity'].unique())
features = train[np.arange(480).astype(str)]
# features = train[train.columns[1:-2]]

In [69]:
features.shape, len(labels), K

((81262, 480), 81262, 2)

In [70]:
# This is a sanity check to make sure TensorFlow is reshaping the features 
# into the 3 channels (x, y, z) correctly.
graph = tf.Graph()
with graph.as_default():
    train_features = tf.placeholder(tf.float32, shape=(None, 480))
    x = tf.reshape(train_features, [-1, 160, 3])
    
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print(sess.run(x, feed_dict={train_features: features}))

[[[-0.81599998  0.93400002 -0.83200002]
  [-0.85500002  0.92199999 -0.87099999]
  [-0.73000002  0.852      -0.87099999]
  ..., 
  [-0.51999998  0.83999997  0.125     ]
  [-0.54699999  0.86299998  0.168     ]
  [-0.60500002  0.88300002  0.17200001]]

 [[-0.41800001  0.215      -0.852     ]
  [ 0.19499999  0.37099999 -0.71100003]
  [ 0.76599997  0.50400001 -0.699     ]
  ..., 
  [ 0.50800002  0.086       0.67199999]
  [ 0.42199999  0.09        0.85500002]
  [ 0.414       0.039       1.01199996]]

 [[ 0.60500002  0.523      -0.64099997]
  [ 0.58200002  0.53500003 -0.64099997]
  [ 0.56300002  0.54699999 -0.64499998]
  ..., 
  [ 0.52700001  0.574      -0.65200001]
  [ 0.50800002  0.57800001 -0.65200001]
  [ 0.5         0.57800001 -0.66000003]]

 ..., 
 [[ 0.125       0.98000002  0.117     ]
  [ 0.125       0.949       0.105     ]
  [ 0.125       0.926       0.082     ]
  ..., 
  [ 0.223       0.96499997  0.102     ]
  [ 0.215       0.96899998  0.102     ]
  [ 0.215       0.96499997  0.098  

In [ ]:
num_epochs = 200
learning_rate = 0.08

activation_func = tf.tanh

graph = tf.Graph()
with graph.as_default():
    # Placeholders for the actual data.
    train_features = tf.placeholder(tf.float32, shape=(None, 480))
    train_labels = tf.placeholder(tf.int32, shape=(None, ))
    
    # Reshaping the data
    x = tf.reshape(train_features, [-1, 160, 3])
    y = tf.one_hot(
        train_labels,
        depth=K
    )
    
    # Convolution 1
    conv1 = tf.layers.conv1d(inputs=x, 
                             filters=8, 
                             kernel_size=8, 
                             padding="same", 
                             data_format="channels_last",
                             activation=activation_func)
    # Max Pooling 1 (reduces samples from 80 --> 31)
    pool1 = tf.layers.max_pooling1d(inputs=conv1,
                                    pool_size=40, 
                                    strides=20,
                                    data_format="channels_last")
    
    # Convolution 2
    conv2 = tf.layers.conv1d(inputs=pool1, 
                             filters=16, 
                             kernel_size=8, 
                             padding="same", 
                             data_format="channels_last",
                             activation=activation_func)
    # Max Pooling 2 (reduces samples from 39 --> 17)
    pool2 = tf.layers.max_pooling1d(inputs=conv2,
                                    pool_size=2, 
                                    strides=1,
                                    data_format="channels_last")
    
    # Flatten the Pooled Data
    pool2_flat = tf.reshape(pool2, [-1, 6 * 16])
    
    # Dense Layer (try adding dropout?)
    dense = tf.layers.dense(inputs=pool2_flat, units=128, activation=activation_func)
    
    # Logits Layer
    logits = tf.layers.dense(inputs=dense, units=K)
    
    # Define loss function and optimizer
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits=logits, 
            labels=y
        )
    )
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    # Define accuracy (so that it can be calculated and printed)
    accuracy = tf.reduce_mean(
        tf.cast(
            tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1)),
            tf.float32
        )
    )
    
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    i = 0
    prev = 0
    for _ in range(num_epochs):
        feed_dict = {
            train_features: features,
            train_labels: labels
        }
        _, _, acc = sess.run(
            [optimizer, loss, accuracy], 
            feed_dict=feed_dict)
        print("Epoch: ", i, " Acc: ", acc*100, " Change: ", (acc-prev)*100)
        i = i + 1
        prev = acc

Epoch:  0  Acc:  52.4033367634  Change:  52.4033367634
Epoch:  1  Acc:  55.7960689068  Change:  3.3927321434
Epoch:  2  Acc:  62.6012146473  Change:  6.80514574051
Epoch:  3  Acc:  66.9648766518  Change:  4.36366200447
Epoch:  4  Acc:  69.1331744194  Change:  2.16829776764
Epoch:  5  Acc:  70.2443957329  Change:  1.11122131348
Epoch:  6  Acc:  70.6603348255  Change:  0.415939092636
Epoch:  7  Acc:  70.7464754581  Change:  0.0861406326294
Epoch:  8  Acc:  70.8080053329  Change:  0.0615298748016
Epoch:  9  Acc:  70.7021713257  Change:  -0.105834007263
Epoch:  10  Acc:  70.5963432789  Change:  -0.105828046799
Epoch:  11  Acc:  70.6295669079  Change:  0.0332236289978
Epoch:  12  Acc:  70.6517219543  Change:  0.022155046463
Epoch:  13  Acc:  70.7021713257  Change:  0.0504493713379
Epoch:  14  Acc:  70.7956969738  Change:  0.0935256481171
Epoch:  15  Acc:  70.9002971649  Change:  0.104600191116
Epoch:  16  Acc:  70.9864377975  Change:  0.0861406326294
Epoch:  17  Acc:  71.1082696915  Change: